In [1]:
import mmcv
from mmcv import Config
from mmseg.apis import multi_gpu_test, single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
cfg = Config.fromfile("configs/custom/uppernet/swin.py")
save_dir = "/opt/ml/test.csv"
checkpoint_path = "work_dirs/swin/best_mIoU_epoch_100.pth"

In [3]:
cfg.seed=2022
cfg.gpu_ids = [1]
cfg.model.train_cfg = None
cfg.data.test.test_mode = True
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2022-12-21 04:06:30,966 - mmseg - INFO - Loaded 819 images


In [4]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: work_dirs/swin/best_mIoU_epoch_100.pth


In [5]:
output = single_gpu_test(model, data_loader) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 21.3 task/s, elapsed: 39s, ETA:     0s

In [6]:
import pandas as pd
import json

In [7]:
import matplotlib.pyplot as plt

In [8]:
with open("/opt/ml/aistage/data/trash_test_mmseg/test.json", "r") as f:
    test = json.load(f)

In [9]:
submission=pd.DataFrame(columns=['image_id','PredictionString'])
arr = [str(i) for i in range(819)]
arr.sort()
for image in test['images']:
    ind=arr.index(str(image['id']))
    out=output[ind]
    
    input_size=512
    output_size=256

    bin_size=2

    out=out.reshape((1,512,512))
    result=out.reshape((1,output_size,bin_size,output_size,bin_size)).max(4).max(2)
    submission = submission.append({"image_id" : image['file_name'], "PredictionString" : " ".join(map(str, result.flatten().tolist()))}, 
                                   ignore_index=True)

In [10]:
print(submission)

                 image_id                                   PredictionString
0    batch_01_vt/0021.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1    batch_01_vt/0028.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2    batch_01_vt/0031.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3    batch_01_vt/0032.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4    batch_01_vt/0076.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
..                    ...                                                ...
814     batch_03/0968.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
815     batch_03/0969.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
816     batch_03/0992.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 ...
817     batch_03/0998.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
818     batch_03/0999.jpg  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...

[819 rows x 2 columns]


In [11]:
submission.to_csv(save_dir, index=False)